In [ ]:
import pandas as pd
# import sklearn Column Transformer and Pipeline
from sklearn.compose import ColumnTransformer
# import sklearn preprocessing classes
from sklearn.preprocessing import OneHotEncoder
# import Simple imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression

In [ ]:
import re

def to_snake_case(name):
    name = name.replace(' ', '_')
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('__([A-Z])', r'_\1', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name)
    return name.lower().strip()


In [ ]:
df = pd.read_csv("telecom_customer_churn.csv")
df = df.rename(columns=dict(zip(df.columns, list(map(to_snake_case, df.columns)))))
df.head()

,customer_id,gender,age,married,number_of_dependents,city,zip_code,latitude,longitude,number_of_referrals,...,payment_method,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,customer_status,churn_category,churn_reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [10]:
train_X = train.drop("y", axis=1)

categorical_columns = train_x.select_dtypes(include='object').columns
numerical_columns = train_x.select_dtypes(include='number').columns
# Create the transformer
transformer = ColumnTransformer([
    ('num_imputer', SimpleImputer(strategy='median'), numerical_columns),
    ('cat_imputer', SimpleImputer(strategy='most_frequent'), categorical_columns),
    ('scaler', StandardScaler(), numerical_columns),
    ('encoder', OneHotEncoder(), categorical_columns)
])
ohe = OneHotEncoder()
transformer.fit_transform(train[["customer_id"]])


KeyboardInterrupt



In [6]:
y = "customer_status"

train, test = train_test_split(
    df, test_size=0.2, random_state=0, stratify=df[y])

assert df[y].isna().any() == False


def build_column_transformer_for_df(train_x: pd.DataFrame) -> ColumnTransformer:
    """Builds a column transformer for a pandas dataframe."""
    # Get the categorical and numerical columns
    categorical_columns = train_x.select_dtypes(include='object').columns
    numerical_columns = train_x.select_dtypes(include='number').columns
    # Create the transformer
    transformer = ColumnTransformer([
        ('num_imputer', SimpleImputer(strategy='median'), numerical_columns),
        ('cat_imputer', SimpleImputer(strategy='most_frequent'), categorical_columns),
        ('scaler', StandardScaler(), numerical_columns),
        ('encoder', OneHotEncoder(sparse_output=False, categories='auto'), categorical_columns)
    ], remainder='passthrough')
    return transformer


def build_sklearn_pipeline(df: pd.DataFrame, y_name:str) -> Pipeline:
    """Builds a sklearn pipeline for churn prediction."""
    # Define the steps

    column_transformer = build_column_transformer_for_df(df.drop(y_name, axis=1))

    steps = [
        ('column_transformer', column_transformer),
        ('pca', PCA()),
        ('logistic', LogisticRegression())
    ]
    # Create the pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline

p = build_sklearn_pipeline(train, y)
p.fit(train.drop(y, axis=1), train[y])

ValueError: could not convert string to float: '1686-STUHN'

In [ ]:
y = "customer_status"

train, test = train_test_split(
    df, test_size=0.2, random_state=0, stratify=df[y])

assert df[y].isna().any() == False


def build_column_transformer_for_df(train_x: pd.DataFrame) -> ColumnTransformer:
    """Builds a column transformer for a pandas dataframe."""
    # Get the categorical and numerical columns
    categorical_columns = train_x.select_dtypes(include='object').columns
    numerical_columns = train_x.select_dtypes(include='number').columns
    # Create the transformer
    transformer = ColumnTransformer([
        ('num_imputer', SimpleImputer(strategy='median'), numerical_columns),
        ('cat_imputer', SimpleImputer(strategy='most_frequent'), categorical_columns),
        ('scaler', StandardScaler(), numerical_columns),
        ('encoder', OneHotEncoder(sparse=False, categories='auto'), categorical_columns)
    ], remainder='passthrough')
    return transformer


def build_sklearn_pipeline(df: pd.DataFrame, y_name:str) -> Pipeline:
    """Builds a sklearn pipeline for churn prediction."""
    # Define the steps

    column_transformer = build_column_transformer_for_df(df.drop(y_name, axis=1))

    steps = [
        ('column_transformer', column_transformer),
        ('label_encoder', LabelEncoder()),
        ('pca', PCA()),
        ('logistic', LogisticRegression())
    ]
    # Create the pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline


def sklearn_gridsearch_using_pipeline(train: pd.DataFrame, y_name:str, n_folds: int = 5) -> GridSearchCV:
    """Performs a grid search using a sklearn pipeline."""
    # Get the pipeline
    pipeline = build_sklearn_pipeline(train, y_name=y_name)

    # define stratiefied shuffle split:
    sss = StratifiedShuffleSplit(
        n_splits=n_folds, test_size=0.2, random_state=0)

    # Define the hyperparameter grid
    param_grid = {
        'pca__n_components': [2, 5, 10, 12, 15, 20, 25],
        'logistic__penalty': ['l1', 'l2'],
        'logistic__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    }

    # Perform the grid search
    grid = GridSearchCV(pipeline, param_grid, cv=sss, n_jobs=-1, scoring="roc_auc", verbose=5)
    grid.fit(train.drop(y, axis=1), train[y])
    # Print the results
    print('Best score:', grid.best_score_)
    print('Best parameters:', grid.best_params_)

    return grid

grid = sklearn_gridsearch_using_pipeline(train, y_name=y)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

clf = grid.best_estimator_["logistic"]

test_predictions = grid.best_estimator_.predict(test.drop(y, axis=1))
cm = confusion_matrix(test[y], test_predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=clf.classes_)

disp.plot()
plt.show()

In [22]:
df['A'] = df['monthlycharge']

# Step 2: Define a function to calculate the instance-dependent cost
def instance_dependent_cost(predicted_label, true_label, A):
    if true_label == 1 and predicted_label == 1:  # True Positive (TP)
        return 0
    elif true_label == 0 and predicted_label == 1:  # False Positive (FP)
        return 2 * A
    elif true_label == 0 and predicted_label == 0:  # True Negative (TN)0
        return 0
    elif true_label == 1 and predicted_label == 0:  # False Negative (FN)
        return 12 * A

# Step 3: Apply the function to calculate the cost for each instance and create the instance-dependent cost matrix
df['instance_cost'] = df.apply(lambda row: instance_dependent_cost(row['predicted_labels'], row['true_labels'], row['A']), axis=1)

# Print the DataFrame with the instance-dependent cost matrix
print(df[['monthlycharge', 'predicted_labels', 'true_labels', 'instance_cost']])

KeyError: 'predicted_labels'

In [258]:
X_train = np.array([[...], [...], ...])  # Replace '...' with your actual features

# Example binary target labels
y_train = np.array([0, 1, 1, 0])  # Replace with your actual target labels

# Create a list to store instance-dependent predicted probabilities (X values)
instance_probs = []

# Train a logistic regression model for each instance and get the predicted probabilities
for i in range(len(X_train)):
    # Select the i-th instance as a single-sample training set
    X_single_instance = X_train[i].reshape(1, -1)
    y_single_instance = y_train[i]

    # Create a logistic regression model
    model = LogisticRegression()

    # Fit the model to the single-instance data
    model.fit(X_single_instance, [y_single_instance])

    # Predict the probability for class 1 (positive class)
    prob_class_1 = model.predict_proba(X_single_instance)[:, 1]

    # Append the predicted probability to the list
    instance_probs.append(prob_class_1[0])

# Convert the list to a numpy array
X_predicted_probs = np.array(instance_probs)

# Now, X_predicted_probs contains the instance-dependent predicted probabilities (X values)
print("Instance-Dependent Predicted Probabilities (X):", X_predicted_probs)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [256]:
def cross_entropy_cost(hypothesis, y_true):
    # Ensure the hypothesis values are clipped to avoid taking the logarithm of zero
    epsilon = 1e-15
    hypothesis = np.clip(hypothesis, epsilon, 1 - epsilon)

    # Calculate the cross-entropy cost
    cost = -np.mean(y_true * np.log(hypothesis) + (1 - y_true) * np.log(1 - hypothesis))
    return cost




In [257]:
y_true = np.array([0, 1, 1, 0])
instance_costs = [cross_entropy_cost(X[i], y_true[i]) for i in range(len(X))]

IndexError: index 4 is out of bounds for axis 0 with size 4